# A/B Test
Generally, A/B tests have two variants, in this case a control and treatment group. We will analyze the converted group to see the effectiveess on treatment. Since treatment is being potentially changed, it is the independent variable. Before going further, here is what the data looks like.

In [1]:
import pandas as pd
import numpy as np
file='/kaggle/input/ab-testing/ab_data.csv'
df=pd.read_csv(file)
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


# Brief Analysis
There is an efficient amount of data (more than 100) of balanced data between teatment and control.

In [2]:
df['group'].value_counts()

treatment    147276
control      147202
Name: group, dtype: int64

# Descriptive Stats

In [3]:
df[['group', 'converted']].describe()

,converted
count,294478.000000
mean,0.119659
std,0.324563
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


# Calculate Conversion for Treatment and Control Groups
What is the conversion rate for both the treatment and control group? This is important, if the treatment is working then the conversion rate should be higher for the treatment group.

In [4]:
control = df[df['group'] == 'control']
control_perc=control['converted'].mean(numeric_only=True).round(4)
print("control effectiveness as percent", control_perc)

treatment = df[df['group']=='treatment']
treatment_perc = treatment['converted'].mean(numeric_only=True).round(4)
print("treatment effectiveness as percent", treatment_perc)

control effectiveness as percent 0.1204
treatment effectiveness as percent 0.1189


# Statistical Significance 
Is the results accurate or was it done by random chance? Generally larger results help with Statistical significance. 

In [5]:
import scipy.stats as stats

# Get the control and treatment group converted as a list:
control_group=list(control['converted'].reset_index(drop=True))
treatment_group=list(treatment['converted'].reset_index(drop=True))

# Set up the hypotheses
# Null Hypothesis (H0): There is no significant difference between the means of the control and treatment groups.
# Alternative Hypothesis (H1): There is a significant difference between the means of the control and treatment groups.
# The test will be a two-tailed test.
alpha = 0.05

# Perform an independent samples t-test
t_statistic, p_value = stats.ttest_ind(control_group, treatment_group)
t_statistic=t_statistic.round(4)
p_value=p_value.round(4)
print(t_statistic, " t_statistic")
print(p_value, " p_value")

# Determine statistical significance
if p_value < alpha:
    print("Reject the null hypothesis.")
    print("There is a significant difference between the control and treatment groups.")
else:
    print("Fail to reject the null hypothesis.")
    print("There is no significant difference between the control and treatment groups.")

1.2369  t_statistic
0.2161  p_value
Fail to reject the null hypothesis.
There is no significant difference between the control and treatment groups.


In [6]:
import statsmodels.api as sm

# Example data for two groups
group1_converted = np.array(treatment_group)
group2_converted = np.array(control_group)

# Perform the proportion test
successes = np.array([group1_converted.sum(), group2_converted.sum()])
trials = np.array([len(group1_converted), len(group2_converted)])
z_score, p_value = sm.stats.proportions_ztest(successes, trials)

# Round to four decimal places
z_score = round(z_score, 4)
p_value = round(p_value, 4)

# Print the results
print(f"Z-score: {z_score}")
print(f"P-value: {p_value}")

Z-score: -1.2369
P-value: 0.2161


**P-value:** 0.2161 shows that the data is not significant. A lower value (one below .05) would of shown the treatment was effective. A higher value close to 1 would mean the data is totally random. Therefore, our value of 0.2161 is more in the middle. 



**Z-score: -1.2369** means that the value is about 1.2 standard deviations below the mean (below since the z-score is negative). 